# Landing Performance Assignment

In [31]:
import math
import qty
import numpy as np
import atmosphere

## Deliverable 1 - Dispatch Landing Chart

### Conditions:
- Dry AND wet runways
- Mass variation from 170 to 230 tons, steps of 10 tons
- Field elevation sea level, up to and including 5000 ft, steps of 1000 ft
- Zero wind

### Aircraft data
- Wingspan, $b = 64.5 m$
- Wing surface, $S = 436.8 m^2$
- Oswald factor, $e = 0.8$
- Parasitic drag coefficient in landing configuration, $C_{D_0} = 0.088$
- Braking coefficient on dry runway, $\mu_{dry} = 0.54$
- Braking coefficient on wet runway, $\mu_{wet} = 0.34$
- Lift coefficient in ground run, $C_{L_{gnd}} = 0.126$
- Drag coefficient in ground run, $C_{D_{gnd}} = 0.213$
- IAS stall at 200 tons with flaps 30, ${IAS}_{stall} = 122 Kts$
- Speed during approach, $v_{ref} = 1.23 \cdot {IAS}_{stall}$
- Speed at touchdown, $v_{td} = 1.18 \cdot {IAS}_{stall}$
- Time of transition phase, $t_{tr} = 1.3 s$
- Speed decay factor of transition phase of landing run, $decay = 0.85$
- Thrust per engine at touchdown, $T = 3400 lbs$

### Assumptions
- Landing flaps 30
- Drag polar is exactly parabolic

Initialize the variables in python:

In [32]:
mass = qty.Mass.M_ton(200)
wingspan = qty.Distance(64.5)
surface = qty.Area(436.8)
oswald = 0.8
cd0_land = 0.088
mu_dry = 0.54
mu_wet = 0.34
cl_gnd = 0.126
cd_gnd = 0.213
ias_stall = qty.Velocity.Kts(122)
ias_ref = qty.Velocity(1.23 * ias_stall)
ias_td = qty.Velocity(1.18 * ias_stall)
transition_time = qty.Time(1.3)
speed_decay = 0.85
thrust_land = qty.Force.Lbf(3400 * 2)


# More assumptions

# Modern aircraft, so flare height = screen height = 50 ft
flare_height = qty.Distance.Ft(50)

# Assume that the flare is decelerating

# Assume ILS approach, 3 degrees fpa
fpa = math.radians(3)

In [35]:
elevation = qty.Distance.Ft(2000)
density = atmosphere.get_density_at_height(elevation)
mass = qty.Mass.M_ton(200)

In [51]:
aspect_ratio = wingspan*2 / surface
cdi_k_const = 1 / (math.pi * oswald * aspect_ratio)
cdi_k_const

1.3472650996616256

In [41]:
weight = mass * 9.81
t_min_d_b = -(weight * math.sin(fpa))

tas_td = ias_td * math.sqrt(1.225 / density)

drag_td = qty.Force(0.5 * density * tas_td**2 * surface * cd_gnd)

t_min_d_c = qty.Force(thrust_land - drag_td)

t_min_d_avg = qty.Force((t_min_d_b + t_min_d_c) / 2)


    
print(t_min_d_avg)

-192496.02 Newtons


In [42]:
def average_tmind(mass, elevation):
    weight = mass * 9.81
    t_min_d_b = -(weight * math.sin(fpa))
    density = atmosphere.get_density_at_height(elevation)
    tas_td = ias_td * math.sqrt(1.225 / density)
    drag_td = qty.Force(0.5 * density * tas_td**2 * surface * cd_gnd)
    t_min_d_c = qty.Force(thrust_land - drag_td)
    t_min_d_avg = qty.Force((t_min_d_b + t_min_d_c) / 2)
    
    return t_min_d_avg

In [49]:
masses = [qty.Mass.M_ton(m) for m in np.arange(170, 240, 10)]
elevations = [qty.Distance.Ft(h) for h in np.arange(0, 6000, 1000)]

for mass in masses:
    for elev in elevations:
        t_min_d_avg = average_tmind(mass, elev)
        
        weight = mass * 9.81
        density_flare = atmosphere.get_density_at_height(elev + flare_height)
        density_td = atmosphere.get_density_at_height(elev)
        
        tas_c = ias_td * math.sqrt(1.225 / density_td)
        tas_b = ias_ref * math.sqrt(1.225 / density)

        
        dist_flare = qty.Distance((weight / (2 * 9.81) * (tas_c**2 - tas_b**2) - weight * flare_height) / t_min_d_avg)
        
        print(mass, elev.ft, density_flare, density_td, tas_c, dist_flare)

170000.00 kilograms 0.0 1.2234329157914896 1.2252256827617731 74.05260114403667 522.30 meters
170000.00 kilograms 1000.0 1.1879971697841951 1.1897500738494131 75.14853069212218 447.08 meters
170000.00 kilograms 2000.0 1.1533521776774813 1.1550658374317087 76.26846298288399 369.08 meters
170000.00 kilograms 3000.0 1.1194856241632507 1.1211606528141522 77.41309659581059 288.17 meters
170000.00 kilograms 4000.0 1.0863853015597187 1.0880223069382609 78.5831555448061 204.21 meters
170000.00 kilograms 5000.0 1.054039109616782 1.0556386941869822 79.77939039266916 117.08 meters
180000.00 kilograms 0.0 1.2234329157914896 1.2252256827617731 74.05260114403667 545.44 meters
180000.00 kilograms 1000.0 1.1879971697841951 1.1897500738494131 75.14853069212218 466.90 meters
180000.00 kilograms 2000.0 1.1533521776774813 1.1550658374317087 76.26846298288399 385.44 meters
180000.00 kilograms 3000.0 1.1194856241632507 1.1211606528141522 77.41309659581059 300.94 meters
180000.00 kilograms 4000.0 1.086385301